# Pipeline : build a classifier
## data processing of the manually annoted dataset
## image processing : extraction of RGB volumes
## classifier based on WND-charm Model
## PCA & RDF
## Analyse False Positives

In [1]:
import numpy as np
import pandas as pd
from functools import partial
import multiprocessing
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as cm

In [2]:
%run ./configure.ipynb

In [3]:
%run ../Connection_to_server.ipynb

## Load the pre-processed Data 

In [4]:
%run ./DF_Pre_Processing_MOINS_Neurons.ipynb

# 1- Pre- processing of Original Data

In [5]:
Data_FRAMES=pd.read_pickle(projectpath+'io/Output/Data_FRAMES.pkl')

# 2- Get volumes with Multiprocessing

In [ ]:
%run ./Get_Partial_Data.ipynb

## Load Normalized volums as numpy array

In [ ]:
filename_M_DF =projectpath+'io/Output/M_DF.npy'

In [ ]:
M_DF_parallel=np.load(filename_M_DF)

# 3- Display the RGB images corresponding to normalized volums

In [ ]:
%run ./Display_RGB_Vol.ipynb

# 4- WND CHARM Method

## 4.1- Convert RGB images to Grayscale

In [ ]:
%run ./RGB2GRAY.ipynb

## Load the gray images as numpy array

In [ ]:
filename_Gray_IM =projectpath+'io/Output/GRAY_IM.npy'

In [ ]:
GRAY_IM_parallel=np.load(filename_Gray_IM)

# Display images by class ( 1 or 2)

# New Data_frame without strange images 

################ essai de predire les cells d un clone

In [ ]:
df_clone=Data_FRAMES[Data_FRAMES.Clone==2].copy()

In [ ]:
df_clone.to_pickle(projectpath+'io/Output/df_clone.pkl')

In [ ]:
df_clone=pd.read_pickle(projectpath+'io/Output/df_clone.pkl')

In [ ]:
df_clone.head(18)

# Get the volums for the new data frame...

In [ ]:
%run ./Get_Partial_Data.ipynb

In [ ]:
filename_M_DF =projectpath+'io/Output/M_DF.npy'

In [ ]:
M_DF_parallel=np.load(filename_M_DF)

In [ ]:
%run ./RGB2GRAY.ipynb

In [ ]:
filename_Gray_IM =projectpath+'io/Output/GRAY_IM.npy'

In [ ]:
GRAY_IM_parallel=np.load(filename_Gray_IM)

## 4.2- Extraction of the WND-CHARM Features

In [ ]:
%run ./Features_Exctraction.ipynb

## Load the extracted Features

In [ ]:
filename_FEATURES =projectpath+'io/Output/FEATURES.npy'

In [ ]:
FEATURES=np.load(filename_FEATURES)

## Etablish one np.array for the Features' Names & another one for Features' Values

In [ ]:
FEAT_NAMES=FEATURES[:,1,:]

In [ ]:
FEAT_VALUES=FEATURES[:,0,:]

In [ ]:
FEAT_VALUES=FEAT_VALUES.astype(float)

In [ ]:
filename_FEATURES_Val =projectpath+'io/Output/FEATURES_Val.npy'

In [ ]:
np.save(filename_FEATURES_Val,FEAT_VALUES)

In [ ]:
FEAT_VALUES=np.load(filename_FEATURES_Val)

# 4.3- Features Post- processing

## A- Get the normalized Features

In [ ]:
%run ./Features_Normalization.ipynb

## Load the Normalized Features values

In [ ]:
filename_FVN =projectpath+'io/Output/FV_N.npy'

In [ ]:
FV_N=np.load(filename_FVN)

# PCA & Random Forest Classification

In [ ]:
%run ./Pipeline_PCA_RANDOM_Forest_Copy1.ipynb

# Imshow specific images to understand non differentiation in PCA model

In [ ]:
sns.reset_orig() 

# process in order to display false predicted images

In [ ]:
def Get_IMG_failed_prediction(Validation_label_,predictions_RFC):

    fail_img_pred_N=np.zeros(Validation_label_.shape[0])
    fail_img_pred_A=np.zeros(Validation_label_.shape[0])
#     fail_img_pred_R=np.zeros(Validation_label_.shape[0])

    for t in range(0,Validation_label_.shape[0] ):
        if ((predictions_RFC[t]==1) & (Validation_label_[t]!=1)):
            fail_img_pred_A[t]=t

        if ((predictions_RFC[t]==2) & (Validation_label_[t]!=2)):
            fail_img_pred_N[t]=t
               
#         if ((predictions_RFC[t]!=0) & (Validation_label_[t]==0) ):
#             fail_img_pred_R[t]=t
        

    fail_img_pred_N=fail_img_pred_N[fail_img_pred_N!=0]
    fail_img_pred_A=fail_img_pred_A[fail_img_pred_A!=0]
#     fail_img_pred_R=fail_img_pred_R[fail_img_pred_R!=0]
#     return(fail_img_pred_N,fail_img_pred_A,fail_img_pred_R)
    return(fail_img_pred_N,fail_img_pred_A)

In [ ]:
(fail_img_pred_Neurons,fail_img_pred_Astrocytes)=Get_IMG_failed_prediction(yr,predictedVAL)

In [ ]:
fail_img_pred_Neurons.shape

In [ ]:
sns.reset_orig() # sns.set()

In [ ]:
def Plot_Failed_IMG(GRAY_IM, fail_IMG):
    fig = plt.figure(figsize=(12, 12))
    fig.subplots_adjust(hspace=0.2, wspace=0.2)
    for i in range(1,23): 
        plt.subplot(6,4, i)
        plt.title(int(fail_IMG[i-1]))
        plt.imshow(GRAY_IM[int(fail_IMG[i-1])])
       

In [ ]:
%matplotlib notebook

In [ ]:
def Get_DF_failed_predicions(Data_frame, fail_img_prediction):
    newDF=pd.DataFrame(data=None, columns=Data_frame.columns)
    for i in range(0, len(fail_img_prediction) ):
        newDF = newDF.append(Data_frame.loc[Data_frame.indice==(fail_img_prediction[i])].copy())
    return(newDF)

In [ ]:
Data_F_fail_Neurons=Get_DF_failed_predicions(Data_FRAMES, fail_img_pred_Neurons)

In [ ]:
Data_F_fail_Astrocytes=Get_DF_failed_predicions(Data_FRAMES, fail_img_pred_Astrocytes)

# test prediction sur  1 clone

In [ ]:
df_clone=Data_FRAMES[Data_FRAMES.Clone==2].copy()

In [ ]:
df_clone.head()

In [ ]:
import numpy as np
import pandas as pd
from functools import partial
import multiprocessing

In [ ]:
GG=pd.read_pickle(projectpath+'io/Output/Data_FRAMES.pkl')

In [ ]:
conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
connected = conn.connect()

imageId = 451
image = conn.getObject("Image", imageId)
pixels = image.getPrimaryPixels() # get raw pixels information

In [ ]:
print image.getName(), image.getDescription()
# Retrieve information about an image.
print " X:", image.getSizeX()
print " Y:", image.getSizeY()
print " Z:", image.getSizeZ()
print " C:", image.getSizeC()
print " T:", image.getSizeT()

In [ ]:
size_x=image.getSizeX()
size_y=image.getSizeY()
size_z = image.getSizeZ()
size_c = image.getSizeC()
size_t = image.getSizeT()
z, t, c = 0, 0, 0
pixels = image.getPrimaryPixels()
plane = pixels.getPlane(z, c, t)      # get a numpy array.


In [ ]:
# Get a Z-stack of tiles. Using getTiles or getPlanes (see below) returns
# a generator of data (not all the data in hand) The RawPixelsStore is
# only opened once (not closed after each plane) Alternative is to use
# getPlane() or getTile() multiple times - slightly slower.
c, t = 0, 0                 # First channel and timepoint
tile = (50, 50, 10, 10)     # x, y, width, height of tile

# list of [ (0,0,0,(x,y,w,h)), (1,0,0,(x,y,w,h)), (2,0,0,(x,y,w,h))... ]
zct_list = [(iz, c, t, tile) for iz in range(3)]
print "\nZ stack of tiles:"
planes = pixels.getTiles(zct_list)
for i, p in enumerate(planes):
    print "Tile:", zct_list[i], " min:", p.min(),\
        " max:", p.max(), " sum:", p.sum()

In [ ]:
updateService = conn.getUpdateService()

In [ ]:
# def get_coordinates(index_ligne,imageId, large):
#     conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
#     connected = conn.connect()

#     image = conn.getObject("Image", imageId)
#     pixels = image.getPrimaryPixels() # get raw pixels information
    
#     size_x=image.getSizeX()
#     size_y=image.getSizeY()
#     size_z = image.getSizeZ()
#     size_c = image.getSizeC()
#     size_t = image.getSizeT()
   
#     # coordinates of the fist tile:
#     delta_x=int(large/2)
    
#     nb_fenetres=np.arange(size_x/delta_x)
#     index=nb_fenetres
    
#     x=(index*delta_x)+delta_x
#     x=x[1:-1]
#     y=((index_ligne*delta_x)+delta_x)*np.ones(len(index))
#     y=y[1:-1]
    
#     Matrix=np.zeros((len(index),large,large,3))
    
#     z=12
    
#     tilewidths = np.ones(len(index),dtype=int)*large
#     tileheights = tilewidths
#     tilestacks = zip(x,y,tilewidths,tileheights)
    
#     zstacks= np.ones(len(index),dtype=int)*z
    
#     tstacks=np.zeros(len(index), dtype=int)
    
#     for c in range(0,3):
#         cstacks=np.ones(len(index), dtype=int)*c
#         liste=zip(zstacks,cstacks,tstacks, tilestacks)
#         pxobj = pixels.getTiles(liste)
#         for i, p in enumerate(pxobj):
#             Matrix[i,:,:,c]=p
#             Norm_Matrix=(Matrix-np.min(Matrix))*1.0/(np.max(Matrix)-np.min(Matrix))
#     M_mean=np.mean(Norm_Matrix, axis=0)
    
#     conn._closeSession()
#     return(M_mean)

### Build a new data frame (nb lines= nb tiles in the whole image)

In [ ]:
len(index)

In [ ]:
def get_coordinates(index_ligne,imageId, large):
    conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
    connected = conn.connect()

    image = conn.getObject("Image", imageId)
    pixels = image.getPrimaryPixels() # get raw pixels information
    
    size_x= image.getSizeX()
    size_y= image.getSizeY()
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
   
    # coordinates of the fist tile:
    delta_x=int(large/2)
    
    nb_fenetres=np.arange(size_x/delta_x)
    index=nb_fenetres
    
    x=(index*delta_x)+delta_x
    x=x[2:-2]
    y=((index_ligne*delta_x)+delta_x)*np.ones(len(x)) #(index))
    y=y[2:-2]
    
    Matrix=np.zeros((len(x),large,large,3))
    
    z=2
    
    tilewidths = np.ones(len(x),dtype=int)*large
    tileheights = tilewidths
    tilestacks = zip(x,y,tilewidths,tileheights)
    
    zstacks= np.ones(len(x),dtype=int)*z
    
    tstacks=np.zeros(len(x), dtype=int)
    
    for c in range(0,3):
        cstacks=np.ones(len(x), dtype=int)*c
        liste=zip(zstacks,cstacks,tstacks, tilestacks)
        pxobj = pixels.getTiles(liste)
        for i, p in enumerate(pxobj):
            Matrix[i,:,:,c]=p
            Norm_Matrix=(Matrix-np.min(Matrix))*1.0/(np.max(Matrix)-np.min(Matrix))
    M_mean=np.mean(Norm_Matrix, axis=0)
    
    conn._closeSession()
    return(M_mean)

In [ ]:
import numpy as np
import pandas as pd
from functools import partial
import multiprocessing

In [ ]:
conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
connected = conn.connect()

image = conn.getObject("Image", imageId)
pixels = image.getPrimaryPixels() # get raw pixels information

size_x=image.getSizeX()
size_y=image.getSizeY()
size_z = image.getSizeZ()
size_c = image.getSizeC()
size_t = image.getSizeT()

imageId=451
zsize=3
large=25
nidX=size_x/int(large/2)

index_ligne=(size_y/int(large/2))
index_ligne


In [ ]:
from multiprocessing import Pool
from functools import partial

In [ ]:
partial_getDATA_coord = partial(get_coordinates, imageId=imageId, large=large)+

In [ ]:
5892+12

In [ ]:
pool = multiprocessing.Pool(25)
M_Mv = pool.map(partial_getDATA_coord, xrange(491))
pool.close()

In [ ]:
partial_getDATA_coord(1)

In [ ]:
liste

In [ ]:
pxobj

In [ ]:
x.shape